In [1]:
from autocode import Optimization

optimization: Optimization = Optimization(
    num_cpus=2,
    server_host="0.0.0.0",
    server_port=10000,
    dashboard_port=10001,
)
optimization.reset()

2024-08-07 22:06:05,402	INFO worker.py:1772 -- Started a local Ray instance. View the dashboard at http://172.25.4.192:8265 


Available resources:  {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'node:172.25.4.192': 1.0, 'CPU': 2.0, 'object_store_memory': 4452416716.0, 'memory': 8904833435.0, 'GPU': 1.0}
Server is healthy.


In [2]:
from python_on_whales import DockerClient
from typing import List

optimization.reset(keys=["clients"])
docker_clients: List[DockerClient] = optimization.deploy(
    compose_files=["./client/docker-compose.yml"],
)

In [16]:
from autocode import OptimizationObjective
from typing import Dict, Any, List
from autocode import OptimizationEvaluateRunResponse

objectives: List[OptimizationObjective] = [
    OptimizationObjective(
        type="minimize",
    ),

    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),

    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
]


def evaluator(inputs: List[OptimizationEvaluateRunResponse]) -> Dict[str, Any]:
    f_gateway: List[float] = []
    f_account: List[float] = []
    f_product: List[float] = []
    for input in inputs:
        if "gateway" in input.get_client().name:
            f_gateway = input.objectives
        elif "account" in input.get_client().name:
            f_account = input.objectives
        elif "product" in input.get_client().name:
            f_product = input.objectives

    output: Dict[str, Any] = {
        "F": f_gateway + f_account + f_product
    }

    return output


In [5]:
result = optimization.run(
    objectives=objectives,
    num_inequality_constraints=0,
    num_equality_constraints=0,
    evaluator=evaluator
)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |     14 |             - |             -
     2 |      200 |     14 |  1.174194E+01 |         nadir
     3 |      300 |     14 |  0.0023041475 |             f
     4 |      400 |     14 |  0.0230414747 |             f
     5 |      500 |     14 |  0.0115207373 |             f
     6 |      600 |     14 |  4.1666666667 |         nadir
     7 |      700 |     14 |  0.2500000000 |         ideal
     8 |      800 |     13 |  0.3333333333 |         nadir
     9 |      900 |     14 |  0.5714285714 |         ideal
    10 |     1000 |     16 |  0.000000E+00 |             f
    11 |     1100 |     17 |  0.0252100840 |             f
    12 |     1200 |     18 |  0.000000E+00 |             f
    13 |     1300 |     18 |  0.0158730159 |             f
    14 |     1400 |     18 |  0.000000E+00 |             f
    15 |     1500 |     18 |  0.0079365079 |             f
    16 |     1600 |     18 |  0.000000E+00 |            